# Fetch books using google api

In [173]:
import requests
import json
import os

In [84]:
api = 'https://www.googleapis.com/books/v1/volumes'
base_folder = './topics'
os.listdir(base_folder)

['Business & Investing',
 'Crafts & Hobbies',
 'Art, Music & Photography',
 'Home & Garden',
 'True Crime',
 'History',
 'Computers & Technology',
 'Travel',
 'Kids',
 'Biographies & Memoirs',
 'Horror',
 'Biology',
 'Comics & Graphics Novels']

In [175]:
dir = 'Sports & Outdoors'
folder = base_folder + '/' + dir
if dir not in os.listdir(base_folder):
    os.mkdir(folder) 
catg = ['BasketBall','Camping','Football','Golf','Hockey','Olympics','Soccer','Martial Arts','Boating','Baseball']

In [24]:
def fetch(subject, file, q=0, folder=folder, loop=3):
    arr = []
    api = 'https://www.googleapis.com/books/v1/volumes'
    for l in range(loop):
        if not q:
            url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=' + str(l * 40) + '&maxResults=40'
        else:
            url = api + '?q=' + subject + '&langRestrict=en&startIndex=' + str(l * 40) + '&maxResults=40'
        res = requests.get(url).json()
        if not res['totalItems']:
            break
        arr += res['items']
        if len(res['items']) < 40:
            break
    if len(arr):
        print(len(arr))
        with open(folder + '/' + file + '.json', 'w') as f:
            json.dump(arr, f)

In [245]:
file = catg[9]
print(file)
subject = file

IndexError: list index out of range

In [240]:
# url = 'https://www.googleapis.com/books/v1/volumes?q=subject:' +subject + '&langRestrict=en&maxResults=40'
url = 'https://www.googleapis.com/books/v1/volumes?q=' +subject + '&langRestrict=en&maxResults=40'
res = requests.get(url).json()
print(res['totalItems'])
if res['totalItems']:
    res = res['items']
    for i in range(0, min(40, len(res))):
        print('title:' + res[i]['volumeInfo']['title'])
        if 'categories' in res[i]['volumeInfo']:
            print('category: ' + str(res[i]['volumeInfo']['categories']))
        if 'description' in res[i]['volumeInfo']:
            print('description: ' + res[i]['volumeInfo']['description'][:200])
        print()

3139
title:Encyclopedia of Major League Baseball Clubs
category: ['Baseball teams']

title:Baseball
category: ['Social Science']
description: In Baseball: The People's Game, Dorothy Seymour Mills and Harold Seymour produce an authoritative, multi-volume chronicle of America's national pastime. The first two volumes of this study -The Early 

title:Baseball
category: ['Sports & Recreation']
description: A succinct history of baseball, newly revised and updated

title:Baseball
category: ['Social Science']
description: Now available in paperback, Harold Seymour and Dorothy Seymour Mills' Baseball: The Early Years recounts the true story of how baseball came into being and how it developed into a highly organized bus

title:Baseball
category: ['Juvenile Nonfiction']
description: Presents the mathematical concepts involved with the sport of baseball.

title:Baseball Economics
category: ['Business & Economics']
description: Addresses problems confronting the baseball industry, especially tho

## Fetch Function

In [187]:
fetch(subject, file, q=0, loop=3)

120


## Fetching Manually

In [241]:
arr = []
# url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=0&maxResults=40'
url = api + '?q=' + subject + '&langRestrict=en&startIndex=0&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

40

In [242]:
# url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=40&maxResults=40'
url = api + '?q=' + subject + '&langRestrict=en&startIndex=40&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

80

In [243]:
# url = api + '?q=subject:' + subject + '&langRestrict=en&startIndex=80&maxResults=40'
url = api + '?q=' + subject + '&langRestrict=en&startIndex=80&maxResults=40'
res = requests.get(url).json()
arr += res['items']
len(arr)

120

In [244]:
with open(folder + '/' + file + '.json', 'w') as f:
    json.dump(arr, f)

# Json to CSV

In [7]:
import os
import json
import csv

base_folder = './topics'
book_list = []

label = 0

for folder in os.listdir(base_folder):
    folder_path = os.path.join(base_folder, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        with open(file_path) as f:
            books = json.load(f)
            for book in books:
                book = book['volumeInfo']
                title = book['title'] if 'title' in book else ""
                description = book['description'] if 'description' in book else ""
                if 'authors' in book:
                    authors = ''
                    for author in book['authors']:
                        authors += author + ", "
                    authors = authors.strip(', ')
                else:
                    authors = ''
                text = title + " " + authors + " " + description
                text = text.strip(' ')
                book_list.append({'label': label, 'category': folder + ' ' + file.split('.')[0], 'text': text })
        label += 1

with open('book_dataset.csv', 'w') as f:
    w = csv.DictWriter(f, book_list[0].keys())
    w.writeheader()
    w.writerows(book_list)